# MNIST

아래의 code는 google colab 에서 시험된 결과이다.
Anaconda 에서 제공되는 jupyter notebook에서는 작동이 안될수 있다.
추천하는 방법은 notebook이 아니라 python 파일 자체를 실행하라는 것이다. TPU 예제만 빼고는 anaconda prompt 에서 *.py 파일을 실행하는 방법을 추천한다.
compare 도구로서는 "BCcompare"를 추천한다. working day로 30일간 무료로 사용이 가능하다.
무료 사용후 재 설치 하면 다시 30 working day동안 사용이 가능하다.

Compare 도구를 사용하여, from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model, Sequential
 와 "11_TF2_MNIST_sparse_network_modify.py"를 비교하여 차이가 나는 부분을 살펴 보자.

Dense/Flatten, Conv2D layer 등은 아래와 같이 정의되어 질수도 있고, 

```
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model, Sequential
```
"01_TF2_MNIST_beginner_sparse.py" 와 같이 Network 부분에서 정의 되어 질술도 있다.
이는 프로그램을 만드는 사람이 편한부분에서 정의하면 된다.


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model, Sequential


"01_TF2_MNIST_beginner_sparse.py" 와 비교하여 다음 부분이 추가된것을 볼수 있다.

```
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]
```
MNIST 과 같은 gray color 로 지정된 데이터 셋은 [image_size, image_size, 1]로 tensor의 형태가 변환되어야 한다.
tensor의 변환전의 형태와 변환후의 형태를 출력해보자.
RGB color는 [image_size, image_size, 3]형태로 되어야 한다. Cifar10이나 cifar100 의 경우는 [image_size, image_size, 3]로 되어 있으므로 형태 변환이 필요 없다. 

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

mnist = tf.keras.datasets.mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

print(Y_train[0:10])

print(X_train.shape)

# in the case of Keras or TF2, type shall be [image_size, image_size, 1]
# if it is RGB type, type shall be [image_size, image_size, 3]
# For MNIST or Fashion MNIST, it need to reshape
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

print(X_train.shape)

11493376/11490434 [==============================] - 1s 0us/step
[5 0 4 1 9 2 1 3 1 4]
(60000, 28, 28)
(60000, 28, 28, 1)


"01_TF2_MNIST_beginner_sparse.py" 와 비교하였을때, tf.keras.layers.가 사라져 있을 알수 있다.
```
model = Sequential([
    Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME',input_shape=(28, 28, 1)),
    MaxPool2D(padding='SAME'),
    Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'),
    MaxPool2D(padding='SAME'),
    Conv2D(filters=256, kernel_size=3, activation=tf.nn.relu, padding='SAME'),
    MaxPool2D(padding='SAME'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])
```
이는 프로그램 시작부분에서 tf.keras.layers 에서 Conv2D, MAxPool2D 등을 import 하였기 때문이다.

In [3]:

model = Sequential([
    Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME',input_shape=(28, 28, 1)),
    MaxPool2D(padding='SAME'),
    Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'),
    MaxPool2D(padding='SAME'),
    Conv2D(filters=256, kernel_size=3, activation=tf.nn.relu, padding='SAME'),
    MaxPool2D(padding='SAME'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])




```
model.summary()
```
는 network의 구조를 정의 하는 부분이다. 이를 실행하면 아래와 같이 network의 구조를 print한다.


In [4]:

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 256)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

나머지 부분은 동일하다.

In [5]:

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=5, verbose=1)

model.evaluate(X_test,  Y_test, verbose=2)              


Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1202 - accuracy: 0.9622
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0406 - accuracy: 0.9886
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0296 - accuracy: 0.9912
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0220 - accuracy: 0.9930
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0186 - accuracy: 0.9941
313/313 - 1s - loss: 0.0211 - accuracy: 0.9935


[0.021073808893561363, 0.9934999942779541]